In [ ]:
import os
print(os.getcwd())

In [ ]:
from pre_sal_ii.improc import colorspace

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from pre_sal_ii.improc import scale_image_and_save, adjust_gamma

In [ ]:
from importlib import reload
import pre_sal_ii.improc.custom as custom
import localizable_resources as lr

def reload_libs_env():
    from dotenv import load_dotenv
    load_dotenv(".env", override=True)

    reload(custom)
    reload(lr)

reload_libs_env()

In [ ]:
image_name = "ML-tste_original"
path = f"../data/classificada_01/{image_name}.jpg"
scale_image_and_save(path, "../out/classificada_01/", 25)

image_name = "ML-tste_classidicada"
path = f"../data/classificada_01/{image_name}.jpg"
scale_image_and_save(path, "../out/classificada_01/", 25)

In [ ]:
image_name = "ML-tste_original"
path = f"../out/classificada_01/{image_name}_25.jpg"
inputImage = cv2.imread(path)
gamma = 0.5
inputImage = adjust_gamma(inputImage, gamma)

We need a binarized image to locate positions in the image that will be used as training data. I.e. only regions near white pixels will be used in the training process.

In [ ]:
image_pores = custom.proc_pores_basic(inputImage, gamma=None)

In [ ]:
plt.imshow(image_pores, cmap='gray')
plt.axis('off')

We now identify areas in which we have moldic pores as labeled by Julia Favoreto.

In [ ]:
image_name = "ML-tste_classidicada"
path = f"../out/classificada_01/{image_name}_25.jpg"
binaryImage_clRed = custom.proc_moldic_pores(path)
plt.imshow(binaryImage_clRed, cmap='gray')
cv2.imwrite("../out/binaryImage_clRed.jpg", binaryImage_clRed)

In [ ]:
import pre_sal_ii.models.ds as ds
reload(ds)

In [ ]:
from torch.utils.data import DataLoader
from pre_sal_ii.models.features import build_feature_stack
image_prob_map = image_pores
input_features = build_feature_stack(inputImage)
num_samples = 10000
dataset = ds.ProbabilityMapPixelRegionDataset(
    image_prob_map, input_features, binaryImage_clRed/255., num_samples=num_samples,
    region_size=101, target_region_size=1)

In [ ]:
max(dataset[0][0].flatten())

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
for it, (img, imgTarget, point) in enumerate(dataset):
    if it >= 10: break
    print(img.permute(1, 2, 0).shape)
    img = img.permute(1, 2, 0)[:,:,0:3]
    imgTarget = imgTarget.permute(1, 2, 0)
    # print(img.numpy().shape, img.dtype)
    axes[it//5*2+0, it%5].imshow(img.numpy(), cmap="gray", vmin=0, vmax=1)
    axes[it//5*2+1, it%5].imshow(imgTarget.numpy(), cmap="gray", vmin=0, vmax=1)

In [ ]:
print(f"len(dataset) = {len(dataset)}")
print(f"dataset[0][0].shape = {dataset[0][0].shape}")
print(f"dataset[0][1].shape = {dataset[0][1].shape}")
print(f"max(dataset[0][0].flatten()) = {max(dataset[0][0].flatten())}")
print(f"max(dataset[0][1].flatten()) = {max(dataset[0][1].flatten())}")

In [ ]:
from torch.utils.data import DataLoader
dl = DataLoader(dataset, batch_size=32, shuffle=False)

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

import torch.nn as nn
import pre_sal_ii.models.nn as nn_models
reload(nn_models)
use_features = 3
model = nn_models.EncoderNN(initial_dim=use_features*32*32).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

print(device)
model

In [ ]:
for k in dl:
    print(k[0].shape)
    break

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import copy

best_model_state = None
best_model_loss = 0

interpolate = lambda x: F.interpolate(
    x, size=(32, 32), mode='bilinear', align_corners=False)
# TODO: try with lanczos ?

num_epochs = 100

from tqdm import tqdm
bar = tqdm(total=num_epochs*num_samples)

for epoch in range(20):
    for features, gray_targets, _ in dl:
        features = features[:,0:use_features,:,:]
        # assert ((*features.shape[1:],) == (use_features, 101, 101))
        features = interpolate(features.to(device)).view(-1, use_features*32*32)
        # assert ((*features.shape[1:],) == (use_features*32*32,))
        # assert (max(features.flatten()) <= 1.0)
        
        # assert ((*gray_targets.shape[1:],) == (1, 1, 1))
        gray_targets = gray_targets.to(device)
        # assert (max(gray_targets.flatten()) <= 1.0)
        # assert ((*gray_targets.shape[1:],) == (1, 1, 1))

        preds = model(features)
        loss = criterion(preds, gray_targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if best_model_state is None or loss.item() < best_model_loss:
            best_model_state = copy.deepcopy(model.state_dict())
            best_model_loss = loss.item()

        bar.update(32)

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
    

In [ ]:
num_samples = 20
dataset2 = ProbabilityMapPixelRegionDataset(
    image_prob_map, input_features, None, num_samples=num_samples,
    region_size=11)

from tqdm import tqdm
with torch.no_grad():
    for it, (features, _, coords) in enumerate((dataset2)):
        inputs = interpolate(features.to(device).unsqueeze(0))
        features = inputs.view(-1, 33*32*32)
        assert ((*features.shape,) == (1, 33*32*32))
        Y = model(features)
        assert ((*Y.shape,) == (1, 1))

        sz = 50
        x, y = coords[0], coords[1]
        should_be = binaryImage_clRed[y-sz:y+sz+1, x-sz:x+sz+1]

        if max(should_be.flatten()) < 255:
            continue
        
        plt.figure(figsize=(2, 6))
        plt.subplot(1, 3, 1)
        plt.imshow(inputs[0, 0:3].squeeze(0).permute(1, 2, 0).cpu().numpy(), cmap="gray", vmin=0, vmax=1)
        plt.subplot(1, 3, 2)
        plt.imshow(should_be)
        plt.subplot(1, 3, 3)
        plt.imshow(Y.cpu().numpy(), cmap="gray", vmin=0, vmax=1)
        plt.show()